# Кросс-валидация 

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Пример данных

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [17]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

In [18]:
###############################################################################################
#### Вызываем SPLIT дважды! Здесь мы создаём три набора данных - TRAIN, VALIDATION и TEST  #########
###############################################################################################
from sklearn.model_selection import train_test_split

# 70% данных определяем в обучающий набор, остальные 30% откладываем в сторону
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Оставшиеся 30% разбиваем на оценочный и тестовый наборы данных
# Каждый будет по 15% от исходного набора данных 
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [19]:
# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Создаём модель**

In [20]:
from sklearn.linear_model import Ridge

In [21]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [22]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [23]:
y_eval_pred = model.predict(X_eval)

**Оценка модели (Evaluation)**

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823872

**Уточняем параметры и повторно оцениваем модель**

In [26]:
model = Ridge(alpha=1)

In [27]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [28]:
y_eval_pred = model.predict(X_eval)

**Ещё раз оцениваем модель**

In [29]:
mean_squared_error(y_eval,y_eval_pred)

2.3837830750569866

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [30]:
y_final_test_pred = model.predict(X_test)

In [31]:
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

## Кросс-валидация с помощью cross_val_score


In [32]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [33]:
model = Ridge(alpha=100)

In [34]:
from sklearn.model_selection import cross_val_score

In [35]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [36]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [37]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

8.215396464543607

**Уточняем модель на основе метрик**

In [38]:
model = Ridge(alpha=1)

In [39]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [40]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

3.344839296530695

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [41]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [42]:
y_final_test_pred = model.predict(X_test)

In [43]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514

# Кросс-валидация с помощью cross_validate

Функция cross_validate отличается от cross_val_score двумя аспектами:

эта функция позволяет использовать для оценки несколько метрик;

она возвращает не только оценку на тестовом наборе (test score), но и словарь с замерами времени обучения и скоринга, а также - опционально - оценки на обучающем наборе и объекты estimator.

В случае одной метрики для оценки, когда параметр scoring является строкой string, вызываемым объектом callable или значением None, ключи словаря будут следующими:
        
        - ['test_score', 'fit_time', 'score_time']

А в случае нескольких метрик для оценки, возвращаемый словарь будет содержать следующие ключи:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score по умолчанию принимает значение False, чтобы сэкономить вычислительные ресурсы. Чтобы посчитать оценки на обучающем наборе, достаточно установить этот параметр в значение True.

In [44]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
model = Ridge(alpha=100)

In [46]:
from sklearn.model_selection import cross_validate

In [47]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [48]:
scores

{'fit_time': array([0.00200057, 0.        , 0.00099993, 0.00100064, 0.00100017]),
 'score_time': array([0.        , 0.00099993, 0.00099993, 0.        , 0.        ]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [49]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.002001,0.000,-2.312430,-9.325530,-6.449885
1,0.000000,0.001,-1.746534,-4.944962,-5.589261
2,0.001000,0.001,-2.562117,-11.396652,-10.339140
3,0.001001,0.000,-2.018732,-7.024211,-6.619504
4,0.001000,0.000,-2.279519,-8.385627,-7.755785


In [50]:
pd.DataFrame(scores).mean()

fit_time                        0.001000
score_time                      0.000400
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Уточняем модель на основе метрик**

In [51]:
model = Ridge(alpha=1)

In [52]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [53]:
pd.DataFrame(scores).mean()

fit_time                        0.000600
score_time                      0.000000
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [54]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [55]:
y_final_test_pred = model.predict(X_test)

In [56]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514